In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, LeakyReLU, Flatten, Dropout, BatchNormalization,Add,GlobalAveragePooling2D,Softmax
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
from sklearn.model_selection import train_test_split
import random

In [ ]:
train_datas = pd.read_csv("../input/Kannada-MNIST/train.csv")
val_datas = pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")

In [ ]:
print(train_datas.shape)
print(val_datas.shape)
#train_datas = train_datas / 255.0
#val_datas = val_datas / 255.0

#concat train and val
datas = pd.concat([train_datas,val_datas],axis=0)
print(datas.shape)
datas_X = np.array(datas.drop("label",axis=1),dtype=np.float32)
datas_Y = np.array(datas[["label"]],dtype=np.int32)
train_X,val_X,train_Y,val_Y = train_test_split(datas_X,datas_Y,test_size=0.1,shuffle=True)

In [ ]:
train_X = np.reshape(train_X,(-1,28,28,1))
val_X = np.reshape(val_X,(-1,28,28,1))

print(train_X.shape,train_X.dtype)
print(train_Y.shape,train_Y.dtype)
print(val_X.shape,val_X.dtype)
print(val_Y.shape,val_Y.dtype)

In [ ]:
# models_blocks = [
#     [[64,128,2],[128,256,4],[256,10,2]]
# ]
# CLASS_NUMS = 10
# #####
# tf.reset_default_graph()
# nets = len(models_blocks)
# model = [0] * nets
# for j,blocks in enumerate(models_blocks):
#     input_layer = Input(shape=(28,28,1))
#     net = Conv2D(64, (3,3), padding='same')(input_layer)
#     net = BatchNormalization(momentum=0.95, epsilon=1e-5, gamma_initializer="uniform")(net)
#     net = LeakyReLU(alpha=0.1)(net)
#     net = Conv2D(64, (3,3), padding='same')(input_layer)
#     net = BatchNormalization(momentum=0.95, epsilon=1e-5, gamma_initializer="uniform")(net)
#     net = LeakyReLU(alpha=0.1)(net)
    
#     for block in blocks:
#         net = Conv2D(block[1], (3,3) ,padding='same')(net)
#         net = BatchNormalization(momentum=0.95, epsilon=1e-5, gamma_initializer="uniform")(net)
#         if len(block) <= 3 or block[3]==True:
#             net = MaxPooling2D(2,2)(net)
#         net = LeakyReLU(alpha=0.1)(net)
#         shortcut = net
#         for count in range(block[2]):
#             net = Conv2D(block[0], (3,3), padding='same')(net)
#             net = BatchNormalization(momentum=0.95, epsilon=1e-5, gamma_initializer="uniform")(net)
#             net = LeakyReLU(alpha=0.1)(net)
#             net = Conv2D(block[1], (3,3), padding='same')(net)
#             net = BatchNormalization(momentum=0.95, epsilon=1e-5, gamma_initializer="uniform")(net)
#             net = LeakyReLU(alpha=0.1)(net)
#         net = Add()([net,shortcut])
# #         if len(block) > 3:
# #             net = Dropout(block[3])(net)
# #     net = Conv2D(CLASS_NUMS, (3,3), padding='same')(net)
# #     net = BatchNormalization(momentum=0.95, epsilon=1e-5, gamma_initializer="uniform")(net)
# #     net = LeakyReLU(alpha=0.1)(net)
#     logits = GlobalAveragePooling2D()(net)
#     output = Softmax()(logits)
    
#     model[j] = Model(inputs=input_layer,outputs=output)
#     model[j].compile(optimizer='adam', loss="sparse_categorical_crossentropy",metrics=["accuracy"])
# print(model[0].summary())

In [ ]:
tf.reset_default_graph()
nets = 1
model = [0]*nets
for j in range(nets):
    input_layer = Input(shape=(28,28,1))
    net = Conv2D(64, (3,3), padding='same')(input_layer)
    net = BatchNormalization(momentum=0.5, epsilon=1e-5, gamma_initializer="uniform")(net)
    net = LeakyReLU(alpha=0.1)(net)
    net = Conv2D(64,  (3,3), padding='same')(net)
    net = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net)

    net = LeakyReLU(alpha=0.1)(net)

    net = MaxPooling2D(2, 2)(net)

    net = Conv2D(128, (3,3), padding='same')(net)
    net = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net)
    net = LeakyReLU(alpha=0.1)(net)
    net = Conv2D(128, (3,3), padding='same')(net)
    net = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net)

    net = LeakyReLU(alpha=0.1)(net)

    net = MaxPooling2D(2,2)(net)
    net = Dropout(0.2)(net) 

    net = Conv2D(256, (3,3), padding='same')(net)
    net = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net)
    net = LeakyReLU(alpha=0.1)(net)

    net = Conv2D(256, (3,3), padding='same')(net)
    net = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net)
    net = LeakyReLU(alpha=0.1)(net)


    net = MaxPooling2D(2,2)(net)
    net = Dropout(0.2)(net)
    
    net = GlobalAveragePooling2D()(net)
    
    net = BatchNormalization()(net)
    output = Dense(10, activation='softmax')(net)
    model[j] = Model(inputs=input_layer,outputs=output)
    model[j].compile(optimizer='Adam', loss="sparse_categorical_crossentropy",metrics=["accuracy"])
print(model[0].summary())

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.25,
    height_shift_range = 0.25,
    shear_range = 0.1,
    zoom_range = 0.25,
    horizontal_flip = False,
    rescale=1/255.0)

In [ ]:
#val_datagen = ImageDataGenerator(rescale=1/255.0)

In [ ]:
annealer = LearningRateScheduler(lambda x: 0.002 * 0.99 ** x)
model_checks = []
for j in range(nets):
    model_checks.append(ModelCheckpoint("model-{}.ckpt".format(j), monitor='val_acc', verbose=1, save_best_only=True,))
early_stop = EarlyStopping(monitor='val_acc', verbose=2, patience=30)

In [ ]:
history = [0] * nets
epochs = 150
batch_size = 1024
for j in range(nets):
    history[j] = model[j].fit_generator(datagen.flow(train_X,train_Y, batch_size=batch_size),
                                        epochs = epochs, 
                                        steps_per_epoch = math.ceil(train_X.shape[0]/batch_size)*2,
                                        validation_data = (val_X/255.0,val_Y),
                                        callbacks=[annealer,model_checks[j],early_stop],
                                        verbose=2)
    print("Train: Model{:d}: Epochs={:d},LearningRate={:.12f} ,Train accuracy={:.5f}, Validation accuracy={:.5f}".format(
        j+1,epochs,tf.keras.backend.get_value(model[j].optimizer.lr),max(history[j].history['acc']),max(history[j].history['val_acc'])))

In [ ]:
test_csv = pd.read_csv("../input/Kannada-MNIST/test.csv")
data_test = np.array(test_csv.drop("id",axis=1),dtype=np.float32)
data_test = np.reshape(data_test,(-1,28,28,1))
X_test = data_test / 255.0
#X_test = data_test
print(X_test.shape,X_test.dtype)

In [ ]:
# for j in range(nets):
#     model[j].load_weights("model-{}.ckpt".format(j))

In [ ]:
# ENSEMBLE
# results = np.zeros( (X_test.shape[0],10))
# for j in range(nets):
#     results = results + model[j].predict(X_test)
# results = np.argmax(results,axis = 1)
# results = results[:,None]

In [ ]:
# x_test1,x_nouse,y_test1,y_nouse = train_test_split(X_test,results,test_size=0.05,shuffle=True)
# new_X = np.concatenate([train_X,val_X,x_test1],axis=0)
# new_Y = np.concatenate([train_Y,val_Y,y_test1],axis=0)
# print(new_X.shape)
# print(new_Y.shape)
# new_train_X,new_val_X,new_train_Y,new_val_Y = train_test_split(new_X,new_Y,test_size=0.1,shuffle=True)
# print(new_train_X.shape)
# print(new_train_Y.shape)
# print(new_val_X.shape)
# print(new_val_Y.shape)

In [ ]:
# history = [0] * nets
# epochs = 30
# batch_size = 128
# for j in range(nets):
#     history[j] = model[j].fit_generator(datagen.flow(new_train_X,new_train_Y, batch_size=batch_size),
#         epochs = epochs, steps_per_epoch = math.ceil(new_train_X.shape[0]*1.0/batch_size),  
#         validation_data = (new_val_X,new_val_Y), callbacks=[model_checks[j],annealer],verbose=1)
#     print("Train: Model{:d}: Epochs={:d},LearningRate={:.12f} ,Train accuracy={:.5f}, Validation accuracy={:.5f}".format(
#         j+1,epochs,tf.keras.backend.get_value(model[j].optimizer.lr),max(history[j].history['acc']),max(history[j].history['val_acc'])))

In [ ]:
for j in range(nets):
    model[j].load_weights("model-{}.ckpt".format(j))

In [ ]:
# ENSEMBLE
results = np.zeros( (X_test.shape[0],10))
for j in range(nets):
    results = results + model[j].predict(X_test)
results = np.argmax(results,axis = 1)

In [ ]:
submission = pd.read_csv("../input/Kannada-MNIST/sample_submission.csv")
submission['label'] = results
submission.to_csv("submission.csv",index=False)